In [19]:
#Example script. Solve one HII region with nH=100 cm^-3, Z=0.1Z_sun, T4=1.
#Assume the incident stellar radiation spectrum shape is given by Conroy et al. (2010), with Chabrier IMF.
#Assume stellar population age of 6 Myr, metallicity of 0.1Z_sun.
#Assume the incident spectrum hydrogen ionizing photon generation rate is 10^50 s^-1.
from __future__ import division
import h5py
import numpy as np
from HIILines import *

#Input parameters
agestar      = 6                                   #Myr
logZstar     = -1                                  #log10(Z_sun)
logQ         = 50                                  #log10(s^-1)
lognH        = 2                                   #log10(cm^-3)
logZgas      = -1                                  #log10(Z_sun)
T4           = 1

#Load incident spectrum. Replace this with your own spectrum.
#The spectrum lookup table can be downloaded from https://zenodo.org/record/6338462#.ZBENinbMKbh
fSEDPath     = '/data001/syang/Cloudy/HIILines/tables/SSP_Spectra_Conroy-et-al_v2.5_imfChabrier.hdf5'
fSEDTable    = h5py.File(fSEDPath,'r')
ageGrid      = fSEDTable['ages'][:]                #Gyr
logZstarGrid = fSEDTable['metallicities'][:]       #log10(Z_sun)
Spec         = fSEDTable['spectra'][:]             #L_sun/Hz
wl           = fSEDTable['wavelengths'][:]         #A
nu           = c*10**13/wl                         #Hz
ageIdx       = np.where(np.abs(agestar-ageGrid*1000)==np.min(np.abs(agestar-ageGrid*1000)))[0]
logZstarIdx  = np.where(np.abs(logZstar-logZstarGrid)==np.min(np.abs(logZstar-logZstarGrid)))[0]
L            = Spec[logZstarIdx,ageIdx,::-1][0]*Lsun2Jps #J/s/Hz
nu           = nu[::-1]

#Solve for OIII and OII region volumes
VOII2VHII, VOIII2VHII = solV(nu,L,logQ,lognH,logZgas,T4)

#Compute HII region line luminosities
INPUT        = [logQ, lognH, logZgas, T4, VOII2VHII, VOIII2VHII]
HIILineL     = assignL(INPUT)

print('HIILines finishes solving this HII region.')
print('log10(OIII 88 micron luminosity/[L_sun]) is:',HIILineL[0])
print('log10(OIII 52 micron luminosity/[L_sun]) is:',HIILineL[1])
print('log10(OIII 4960.3A luminosity/[L_sun]) is:'  ,HIILineL[2])
print('log10(OIII 5008.2A luminosity/[L_sun]) is:'  ,HIILineL[3])
print('log10(OIII 4364.4A luminosity/[L_sun]) is:'  ,HIILineL[4])
print('log10(OII 3727,29A luminosity/[L_sun]) is:'  ,np.log10(10**HIILineL[5]+10**HIILineL[6]))
print('log10(H_beta luminosity/[L_sun]) is:'        ,HIILineL[12])

HIILines finishes solving this HII region.
log10(OIII 88 micron luminosity/[L_sun]) is: 3.2940990817399864
log10(OIII 52 micron luminosity/[L_sun]) is: 3.2113717639057384
log10(OIII 4960.3A luminosity/[L_sun]) is: 3.3202904385233865
log10(OIII 5008.2A luminosity/[L_sun]) is: 3.7870689340205104
log10(OIII 4364.4A luminosity/[L_sun]) is: 1.630095823607185
log10(OII 3727,29A luminosity/[L_sun]) is: 4.005933584419967
log10(H_beta luminosity/[L_sun]) is: 4.096649385303536


In [20]:
#Solving OIII and OII region volumes for a huge number of HII regions can be expensive (simulation post-processing)
#Example script that apply the pre-calculated OIII and OII region volumes look-up table
from __future__ import division
import h5py
import numpy as np
from HIILines import *

#Input parameters
agestar      = 6                                   #Myr
logZstar     = -1                                  #log10(Z_sun)
logQ         = 50                                  #log10(s^-1)
lognH        = 2                                   #log10(cm^-3)
logZgas      = -1                                  #log10(Z_sun)
T4           = 1

#Load incident spectrum. Replace this with your own spectrum.
fSEDPath     = '/data001/syang/Cloudy/HIILines/tables/SSP_Spectra_Conroy-et-al_v2.5_imfChabrier.hdf5'
fSEDTable    = h5py.File(fSEDPath,'r')
ageGrid      = fSEDTable['ages'][:]                #Gyr
logZstarGrid = fSEDTable['metallicities'][:]       #log10(Z_sun)
Spec         = fSEDTable['spectra'][:]             #L_sun/Hz
wl           = fSEDTable['wavelengths'][:]         #A
nu           = c*10**13/wl                         #Hz
ageIdx       = np.where(np.abs(agestar-ageGrid*1000)==np.min(np.abs(agestar-ageGrid*1000)))[0]
logZstarIdx  = np.where(np.abs(logZstar-logZstarGrid)==np.min(np.abs(logZstar-logZstarGrid)))[0]
L            = Spec[logZstarIdx,ageIdx,::-1][0]*Lsun2Jps #J/s/Hz
nu           = nu[::-1]

#Load VOII/VHII lookup table
#The VOII/VHII lookup table can be downloaded from zenedo (to be published).
VOII2VHIITable = h5py.File('/data001/syang/Cloudy/HIILines/tables/VOII2VHII.hdf5')
ageGrid        = VOII2VHIITable['ages'][:]
logZstarGrid   = VOII2VHIITable['metallicities'][:]
lognHGrid      = VOII2VHIITable['lognH'][:]
logQGrid       = VOII2VHIITable['logQ'][:]
logZismGrid    = VOII2VHIITable['logZ'][:]
T4Grid         = VOII2VHIITable['T4'][:]
VOII2VHII      = VOII2VHIITable['VOII2VHII'][:]
VOIII2VHII     = VOII2VHIITable['VOIII2VHII'][:]
VOII2VHIITable.close()

logQIdx        = np.where(logQGrid==logQ)[0]
lognHIdx       = np.where(lognHGrid==lognH)[0]
logZIdx        = np.where(logZgas==logZismGrid)[0]
T4Idx          = np.where(T4==T4Grid)[0]
print(VOIII2VHII[logQIdx,lognHIdx,logZIdx,T4Idx,9,26],VOII2VHII[logQIdx,lognHIdx,logZIdx,T4Idx,9,26])

#A small fraction of VOII/VHII and VOIII/VHII are nan for T4!=1
for i in np.arange(len(T4Grid)):
    idx                          = np.where(np.isfinite(VOII2VHII[:,:,:,i,:,:])==False)
    VOII2VHII[:,:,:,i,:,:][idx]  = VOII2VHII[:,:,:,1,:,:][idx]
    idx                          = np.where(np.isfinite(VOIII2VHII[:,:,:,i,:,:])==False)
    VOIII2VHII[:,:,:,i,:,:][idx] = VOIII2VHII[:,:,:,1,:,:][idx]
VOII2VHIIf     = interpolate.RegularGridInterpolator((logQGrid,lognHGrid,logZismGrid,T4Grid,logZstarGrid,ageGrid*1000),VOII2VHII)
VOIII2VHIIf    = interpolate.RegularGridInterpolator((logQGrid,lognHGrid,logZismGrid,T4Grid,logZstarGrid,ageGrid*1000),VOIII2VHII)

INPUT          = [logQ, lognH, logZgas, T4, logZstarGrid[logZstarIdx][0], ageGrid[ageIdx][0]*1000]
VOIII2VHII     = VOIII2VHIIf(INPUT)[0]
VOII2VHII      = VOII2VHIIf(INPUT)[0]

#Compute HII region line luminosities
INPUT        = [logQ, lognH, logZgas, T4, VOII2VHII, VOIII2VHII]
HIILineL     = assignL(INPUT)

print('HIILines finishes solving this HII region.')
print('log10(OIII 88 micron luminosity/[L_sun]) is:',HIILineL[0])
print('log10(OIII 52 micron luminosity/[L_sun]) is:',HIILineL[1])
print('log10(OIII 4960.3A luminosity/[L_sun]) is:'  ,HIILineL[2])
print('log10(OIII 5008.2A luminosity/[L_sun]) is:'  ,HIILineL[3])
print('log10(OIII 4364.4A luminosity/[L_sun]) is:'  ,HIILineL[4])
print('log10(OII 3727,29A luminosity/[L_sun]) is:'  ,np.log10(10**HIILineL[5]+10**HIILineL[6]))
print('log10(H_beta luminosity/[L_sun]) is:'        ,HIILineL[12])

[0.36473972] [0.61708605]
HIILines finishes solving this HII region.
log10(OIII 88 micron luminosity/[L_sun]) is: 3.293466303459616
log10(OIII 52 micron luminosity/[L_sun]) is: 3.210738985625368
log10(OIII 4960.3A luminosity/[L_sun]) is: 3.319657660243016
log10(OIII 5008.2A luminosity/[L_sun]) is: 3.78643615574014
log10(OIII 4364.4A luminosity/[L_sun]) is: 1.6294630453268146
log10(OII 3727,29A luminosity/[L_sun]) is: 4.006193855497011
log10(H_beta luminosity/[L_sun]) is: 4.096649385303536
